In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta


# Toon alle kolommen in pandas output
pd.set_option('display.max_columns', None)


In [ ]:
# Pad naar bestand (pas aan indien nodig)
pad = r'\studentjourney 10122025.csv'
df = pd.read_csv(pad)

In [108]:
# === 1. INLADEN EN OPSCHONEN ===
df = pd.read_csv(pad)
df=df.sort_values(['deelnemernummer','volgnummer'])
df['begindatum'] = pd.to_datetime(df['begindatum'])
df['einddatum'] = pd.to_datetime(df['einddatum'])
huidige_datum = pd.to_datetime("2025-10-03")
df['einddatum_vul'] = df['einddatum'].fillna(huidige_datum)

# === 2. COMBINATIES & STUDIEJAREN ===
df['dn_bc_niv'] = df['deelnemernummer'].astype(str) + "_" + df['BC_code'].astype(str) + "_" + df['niveau'].astype(str)
# Volgende dn_bc_niv binnen dezelfde deelnemer

# 1. Detecteer wanneer een nieuw blok start
df['block_start'] = (df['dn_bc_niv'] != df['dn_bc_niv'].shift(1)).astype(int)

# 2. Geef elk blok een uniek id
df['block_id'] = df['block_start'].cumsum()

# 4. Schrijf de lengte terug
df['dn_bc_niv'] = df['dn_bc_niv'].astype(str) +'_'+ df['block_id'].astype(str)

# Optioneel: opruimen
df = df.drop(columns=['block_start'])

def bepaal_eerste_1okt(begindatum, einddatum):
    for jaar in range(begindatum.year, einddatum.year + 1):
        okt1 = pd.Timestamp(f"{jaar}-10-01")
        if okt1 > pd.Timestamp("2025-10-01"):
            break
        if begindatum <= okt1 <= einddatum:
            return f"{jaar}/{jaar+1}"
    return np.nan

df['eerste_1oktober_studiejaar'] = df.apply(lambda r: bepaal_eerste_1okt(r['begindatum'], r['einddatum_vul']), axis=1)
df = df[df['eerste_1oktober_studiejaar'].notnull()]
df['cohort_jaar'] = df['eerste_1oktober_studiejaar'].str[:4].astype(int)
df=df[df.cohort_jaar >=2017]
# === 3. ONDERBREKING & SEGMENTATIE ===
df = df.sort_values(['dn_bc_niv', 'begindatum'])
df['vorige_einddatum_vul'] = df.groupby('dn_bc_niv')['einddatum_vul'].shift(1)
df['gat_in_dagen'] = (df['begindatum'] - df['vorige_einddatum_vul']).dt.days
df['onderbreking_dn_bc_niv'] = df['gat_in_dagen'] >= 365
df['segment_id'] = df.groupby(['dn_bc_niv'])['onderbreking_dn_bc_niv'].cumsum().fillna(0).astype(int)
df = df.sort_values(['dn_bc_niv', 'cohort_jaar'])

In [ ]:


# === 4. SEGMENT-INFORMATIE (als één NaT in segment → hele segment NaT) ===
def einddatum_segment_logic(einddatums):
    if einddatums.isna().any():
        return pd.NaT
    return einddatums.max()

segment_info = df.groupby(['dn_bc_niv', 'segment_id']).agg(
    eerste_begindatum_in_segment=('begindatum', 'min'),
    laatste_einddatum_in_segment=('einddatum', einddatum_segment_logic)
).reset_index()

# Merge terug op df
df = df.merge(segment_info, on=['dn_bc_niv', 'segment_id'], how='left')

# Extra: forceer per rij nog NaT als die specifieke rij geen einddatum had
df.loc[df['einddatum'].isna(), 'laatste_einddatum_in_segment'] = pd.NaT

# === 5. INSTROOM ===
df = df.sort_values(['deelnemernummer', 'begindatum'])
df['vorige_dn_bc_niv'] = df.groupby('deelnemernummer')['dn_bc_niv'].shift(1)
df['switch_detected'] = (df['dn_bc_niv'] != df['vorige_dn_bc_niv']) & df['vorige_dn_bc_niv'].notnull()
df['vorige_geslaagd'] = df.groupby('deelnemernummer')['geslaagd'].shift(1)

df['Instroom'] = np.select(
    [
        df.groupby('deelnemernummer').cumcount() == 0,
        df['switch_detected'] & (df['vorige_geslaagd'] == 1),
        df['switch_detected'] & ((df['vorige_geslaagd'] == 0) | df['vorige_geslaagd'].isna())
    ],
    ['Nieuw bij instelling', 'Switch met diploma', 'Switch zonder diploma'],
    default=np.nan
).astype(object)

df.loc[df['onderbreking_dn_bc_niv'] & df['Instroom'].isna(), 'Instroom'] = 'Herstart na tussenjaar'

# Bepaal max(geslaagd) per segment
geslaagd_per_segment = (
    df.groupby(['dn_bc_niv', 'segment_id'])['geslaagd']
    .max()
    .reset_index()
    .rename(columns={'geslaagd': 'geslaagd_segment'})
)

# Merge terug en overschrijf
df = df.merge(geslaagd_per_segment, on=['dn_bc_niv', 'segment_id'], how='left')
df['geslaagd'] = df['geslaagd_segment']
df.drop(columns=['geslaagd_segment'], inplace=True)

df['volgnummer_segment'] = df.groupby(['dn_bc_niv', 'segment_id']).cumcount() + 1
# Filter: alleen eerste regel per unieke combinatie
df = df[df['volgnummer_segment'] == 1].copy()

# === 6. DEDUPLICATIE ===
df['niveau_rank'] = df['niveau'].str.extract('(\d+)').astype(float)
df['duur'] = (df['einddatum_vul'] - df['begindatum']).dt.days
df['herhaal_teller'] = df.groupby(['deelnemernummer', 'dn_bc_niv'])['dn_bc_niv'].transform('count')

df = df.sort_values(['deelnemernummer', 'cohort_jaar', 'herhaal_teller', 'niveau_rank', 'geslaagd', 'duur', 'volgnummer'],
                    ascending=[True, True, False, False, False, False, True])
df = df.drop_duplicates(subset=['deelnemernummer', 'cohort_jaar'], keep='first')
df.drop(columns=['niveau_rank', 'duur', 'herhaal_teller'], inplace=True)

# === 7. UITSTROOM ALGEMEEN LABEL ===
df['laatste_in_dn_bc_niv'] = ~df.duplicated('dn_bc_niv', keep='last')
df['volgende_dn_bc_niv'] = df.groupby('deelnemernummer')['dn_bc_niv'].shift(-1)
df['volgende_geslaagd'] = df.groupby('deelnemernummer')['geslaagd'].shift(-1)

def bepaal_uitstroom(row):
    # Geen geldige einddatum → kan niet uitstroom beoordelen
    if pd.isna(row['einddatum']):
        return 'In opleiding'
    
    # Laatste inschrijving binnen deze combinatie
    if not row['laatste_in_dn_bc_niv']:
        return 'In opleiding'

    # Geen vervolginschrijving bekend
    if pd.isna(row['volgende_dn_bc_niv']):
        return 'Diploma behaald verlaat instelling' if row['geslaagd'] == 1 else 'Verlaat instelling zonder diploma'

    # Volgende inschrijving is andere opleiding (switch)
    if row['dn_bc_niv'] != row['volgende_dn_bc_niv']:
        return 'Diploma behaald switch' if row['geslaagd'] == 1 else 'Switch zonder diploma'

    # Default fallback
    return 'In opleiding'

df['uitstroom_label'] = df.apply(bepaal_uitstroom, axis=1)

# === 8. STUDIEDUUR EN UITSTROOMLABELS OP BASIS VAN SEGMENTDATA ===
df['segment_begindatum'] = df['eerste_begindatum_in_segment']
df['segment_einddatum'] = df['laatste_einddatum_in_segment']#.fillna(df['einddatum_vul'])
df['Segment_volgende_begindatum'] = df.groupby('deelnemernummer')['segment_begindatum'].shift(-1)

df['maandverschil_switch'] = (
    (df['Segment_volgende_begindatum'].dt.year - df['segment_einddatum'].dt.year) * 12
    + (df['Segment_volgende_begindatum'].dt.month - df['segment_einddatum'].dt.month)
)

def tel_studiejaren_segment(begindatum, einddatum):
    jaren = 0
    if pd.isna(begindatum) or pd.isna(einddatum):
        return np.nan
    for jaar in range(begindatum.year, einddatum.year + 1):
        okt1 = pd.Timestamp(f"{jaar}-10-01")
        if begindatum <= okt1 <= einddatum:
            jaren += 1
    return jaren

df['nStudieJaren'] = df.apply(lambda r: tel_studiejaren_segment(r['eerste_begindatum_in_segment'], r['laatste_einddatum_in_segment']), axis=1)
df = df.sort_values(['deelnemernummer', 'cohort_jaar'])
df['laatsteCoh'] = ~df.duplicated('deelnemernummer', keep='last')

labels = ['Diploma behaald verlaat instelling', 'Verlaat instelling zonder diploma',
          'Diploma behaald switch', 'Switch zonder diploma', 'In opleiding']



def uitstroom_label_segment(row, max_jaren):
    if pd.isna(row['segment_einddatum']):  # expliciet checken op NaT
        return 'In opleiding'
    if pd.isna(row['nStudieJaren']):
        return np.nan
    if row['nStudieJaren'] <= max_jaren and row['geslaagd'] == 1 and row['laatsteCoh']:
        return labels[0]
    elif row['nStudieJaren'] <= max_jaren and row['geslaagd'] == 0 and row['laatsteCoh']:
        return labels[1]
    elif row['nStudieJaren'] <= max_jaren and row['geslaagd'] == 1:
        return labels[2]
    elif row['nStudieJaren'] <= max_jaren and row['geslaagd'] == 0:
        return labels[3]
    else:
        return labels[4]  # fallback

for jaar in [1, 2, 3, 4]:
    df[f'{jaar}e jaar'] = df.apply(lambda r: uitstroom_label_segment(r, jaar), axis=1)

# === 9. INSTROOM CORRECTIE VIA 4E JAAR ===
df['vorige_4e_jaar'] = df.groupby('dn_bc_niv')['4e jaar'].shift(1)
df.loc[df['onderbreking_dn_bc_niv'], 'Instroom'] = df.loc[df['onderbreking_dn_bc_niv'], 'vorige_4e_jaar']
df.drop(columns=['vorige_4e_jaar'], inplace=True)

df['DnCohBc'] = df['deelnemernummer'].astype(str) + "_" + df['cohort_jaar'].astype(str) + "_" + df['niveau'] + "_" + df['BC_code']



<>:54: SyntaxWarning: invalid escape sequence '\d'
<>:54: SyntaxWarning: invalid escape sequence '\d'
C:\Users\HMUSS\AppData\Local\Temp\ipykernel_25440\352973524.py:54: SyntaxWarning: invalid escape sequence '\d'
  df['niveau_rank'] = df['niveau'].str.extract('(\d+)').astype(float)


In [ ]:
casestudy = df.copy()
casestudy = casestudy.sort_values(['deelnemernummer', 'volgnummer_segment'])

# Initialiseer kolommen
casestudy[['Voorstroom_team', 'Voorstroom_college', 'Voorstroom_niveau', 'Voorstroom_BC_code']] = "-"
casestudy[['Nastroom_team', 'Nastroom_college', 'Nastroom_niveau', 'Nastroom_BC_code', 'succes_bij_switch']] = "-"

# Groepeer per student
for deelnemer, groep in casestudy.groupby('deelnemernummer'):
    groep = groep.sort_values('volgnummer_segment')
    for i in range(len(groep)):
        idx = groep.index[i]

        # Voorstroom
        if i > 0:
            prev_idx = groep.index[i - 1]
            casestudy.loc[idx, 'Voorstroom_team'] = casestudy.loc[prev_idx, 'Team']
            casestudy.loc[idx, 'Voorstroom_college'] = casestudy.loc[prev_idx, 'College']
            casestudy.loc[idx, 'Voorstroom_niveau'] = casestudy.loc[prev_idx, 'niveau']
            casestudy.loc[idx, 'Voorstroom_BC_code'] = casestudy.loc[prev_idx, 'BC_code']

        # Nastroom
        if i < len(groep) - 1:
            next_idx = groep.index[i + 1]
            casestudy.loc[idx, 'Nastroom_team'] = casestudy.loc[next_idx, 'Team']
            casestudy.loc[idx, 'Nastroom_college'] = casestudy.loc[next_idx, 'College']
            casestudy.loc[idx, 'Nastroom_niveau'] = casestudy.loc[next_idx, 'niveau']
            casestudy.loc[idx, 'Nastroom_BC_code'] = casestudy.loc[next_idx, 'BC_code']
            label = casestudy.loc[next_idx, '4e jaar']
            if pd.isna(label):
                casestudy.loc[idx, 'succes_bij_switch'] = np.nan
            elif label in ['Diploma behaald switch', 'In opleiding', 'Diploma behaald verlaat instelling', 'Switch zonder diploma']:
                casestudy.loc[idx, 'succes_bij_switch'] = 1
            else:
                casestudy.loc[idx, 'succes_bij_switch'] = 0

casestudy['Instroom'] = np.where(casestudy['Instroom'] == 'Diploma behaald switch','Switch met diploma',   casestudy['Instroom'])
casestudy['Instroom'] = casestudy['Instroom'].fillna('Nieuw bij instelling')
casestudy=casestudy[casestudy.Instroom !='In opleiding']

In [88]:
casestudy=casestudy[[
 'eerste_1oktober_studiejaar',
 'deelnemernummer',
 'volgnummer',
 'niveau',
 'BC_code',
 'geslaagd',
 'Team',
 'team_official',
 'College',
 'Cluster',
 'DnCohBc',
 '1e jaar',
 '2e jaar',
 '3e jaar',
 '4e jaar',
  'maandverschil_switch',
 'Instroom',
 'Voorstroom_team',
 'Voorstroom_college',
 'Voorstroom_niveau',
 'Voorstroom_BC_code',
 'Nastroom_team',
 'Nastroom_college',
 'Nastroom_niveau',
 'Nastroom_BC_code',
 'succes_bij_switch']].rename(columns={'eerste_1oktober_studiejaar':'cohort'})

In [89]:
casestudy['cohort'].value_counts()

cohort
2017/2018    11267
2018/2019    10333
2020/2021     9911
2019/2020     9850
2022/2023     9559
2023/2024     9550
2025/2026     9384
2024/2025     9362
2021/2022     9258
Name: count, dtype: int64

In [91]:
casestudy=casestudy[(casestudy.cohort != '2017/2018') &( casestudy.cohort !='2018/2019') & ( casestudy.cohort !='2019/2020') ]

In [92]:
# Exporteren
import datetime
today = datetime.date.today().strftime('%Y%m%d')
output_file = f'casestudy_{today}.xlsx'
casestudy.to_excel(output_file, index=False)
print(f"Bestand opgeslagen als: {output_file}")

Bestand opgeslagen als: casestudy_20251210.xlsx
